In [195]:
import matplotlib.pyplot as plt

In [196]:
import time
import plotly.graph_objects as go



In [197]:
class Vertex:
    def __init__(self,x_coord, y_coord,next_index, prev_index, index):
        self.x = x_coord
        self.y = y_coord
        self.next = next_index
        self.prev = prev_index
        self.index = index


In [198]:
class Polygon:
    def __init__(self, filename):
        self.x_coords = []
        self.y_coords = []
        self.vertex_list = []
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split()
                num_vertices = int(parts[0])
                #print(num_vertices)
                for i in range(num_vertices):
                    x_str, y_str = parts[2 * i + 1], parts[2 * i + 2]
                    num, dem = map(int, x_str.split('/'))
                    x_coord = num / dem
                    self.x_coords.append(x_coord)
                    num, dem = map(int, y_str.split('/'))
                    y_coord = num / dem
                    self.y_coords.append(y_coord)
                    self.vertex_list.append(Vertex(x_coord, y_coord, (i+1)%num_vertices, (i-1)%num_vertices , i))
    
    def plot(self, triangles=None):
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(x=self.x_coords + [self.x_coords[0]], y=self.y_coords + [self.y_coords[0]],
                                 mode='lines+markers', name='Polygon') )
        
        fig.add_trace(go.Scatter(x=self.x_coords, y=self.y_coords, fill='toself',fillcolor='rgba(34, 200, 255, 0.2)',
                                 mode='none'))
        
        fig.update_layout(

            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            showlegend=True,
            hovermode='closest',
            template='plotly_white',
            updatemenus=[{'type': 'buttons',
                          'buttons': [{'label': 'Play',
                                       'method': 'animate',
                                       'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}]},
                                     {'label': 'Pause',
                                      'method': 'animate',
                                      'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}]}]}]
        )
        
        if triangles:
            frames = []
            fig.add_trace(go.Scatter(x=self.x_coords + [self.x_coords[0]], y=self.y_coords + [self.y_coords[0]],
                                mode='lines+markers'))
            for i, triangle in enumerate(triangles):
                tri_x = [triangle[0].x, triangle[1].x, triangle[2].x, triangle[0].x]
                tri_y = [triangle[0].y, triangle[1].y, triangle[2].y, triangle[0].y]
                
                frame = go.Frame(data=[go.Scatter(x=tri_x, y=tri_y, mode='lines', line=dict(color = 'red'),name='Triangle')],
                                 name=f'frame{i}')
                frames.append(frame)
            
            fig.frames = frames
        
        fig.show()
    

        
    def triangulate(self):
        triangles = []
        vertices = self.vertex_list[:]
        
        while len(vertices) > 3:
            for i in range(len(vertices)):
                prev_index = (i - 1) % len(vertices)
                next_index = (i + 1) % len(vertices)
                
                ear_found = self.ear(vertices[prev_index], vertices[i], vertices[next_index],vertices)
                if ear_found:
                    triangles.append((vertices[prev_index], vertices[i], vertices[next_index]))
                    #self.plot(triangles)
                    vertices.pop(i)
                    break
        
        triangles.append((vertices[0], vertices[1], vertices[2]))

        return triangles
    
    def ear(self, prev_vertex, ear_vertex, next_vertex, vertices):
        if self.convex(prev_vertex, ear_vertex, next_vertex):
            for v in vertices:
                if v != prev_vertex and v != ear_vertex and v != next_vertex:
                    if self.inTriangle(v, prev_vertex, ear_vertex, next_vertex):
                        return False
            return True
        return False
    
    def convex(self, prev_vertex, curr_vertex, next_vertex):
        return self.vetorialProduct(prev_vertex, curr_vertex, next_vertex) > 0
    
    def vetorialProduct(self, v1, v2, v3):
        return (v2.x - v1.x) * (v3.y - v1.y) - (v2.y - v1.y) * (v3.x - v1.x)
    

    
    def inTriangle(self, v, v1, v2, v3):
        d1 = self.aux(v, v1, v2)
        d2 = self.aux(v, v2, v3)
        d3 = self.aux(v, v3, v1)
        return not ((d1 < 0 or d2 < 0 or d3 < 0) and (d1 > 0 or d2 > 0 or d3 > 0))
    
    def aux(self, v1, v2, v3):
        return (v1.x - v3.x) * (v2.y - v3.y) - (v2.x - v3.x) * (v1.y - v3.y)
    


In [199]:
input = 'agp2009a-simplerand/randsimple-20-1.pol'
poly = Polygon(input)
poly.plot()
t_i = time.time()
triangulate = poly.triangulate()
t_f = time.time()
poly.plot(triangulate)
print(f"tempo levado: {(t_f-t_i):.4f}")

tempo levado: 0.0010


Referências:

https://www.geeksforgeeks.org/timing-functions-with-decorators-python/
https://www.geometrictools.com/Documentation/TriangulationByEarClipping.pdf
https://github.com/yaugenst/triangulation/tree/master
https://www.w3schools.com/css/css_colors_rgb.asp